## Some information
- Maximum lenght of the brain signals : 919, 906, 594
- Maximum number of chars: 66.

In [31]:
import numpy as np 

from pathlib import Path
import scipy

import matplotlib.pyplot as plt

from torch.utils.data import Dataset

from torch.utils.data import DataLoader

In [164]:
def process_signal(brain_list, block_list):
    return brain_list

def process_text(arr):
    return [str.strip() for str in arr]

def process_file(data_file):
    
    data = scipy.io.loadmat(data_file)
    date = data_file.stem

    n_trials = data['blockIdx'].shape[0]

    date_list = [date for _ in range(n_trials)]

    brain_list = data['spikePow'][0][:]
    block_list =  data['blockIdx'][:]

    brain_list = process_signal(brain_list, block_list)
    
    sentence_list = data['sentenceText']
    sentence_list = process_text(sentence_list)

    return brain_list, sentence_list, date_list


def process_all_files(path):
    
    data_res = {'brain_list':[], 'sentence_list':[], 'date_list':[]}
    
    for data_file in sorted(path.glob('*.mat')):

        brains, sentences, dates = process_file(data_file)
        
        data_res['brain_list'].extend(brains)
        data_res['sentence_list'].extend(sentences)
        data_res['date_list'].extend(dates)

    return data_res

In [165]:


class BrainDataset(Dataset):
    def __init__(self, path): 

        data = process_all_files(path)
            
        self.max_tokens = 64
        self.inputs = data['brain_list']
        self.targets = data['sentence_list']
        self.date = data['date_list']
        
        all_labels = []

        ## IMPORTANT TO PAD INDEXES WITH -100 values. 
        # for text in self.targets: 
            # labels = self.processor.tokenizer(text, padding="max_length", max_length=self.max_tokens).input_ids
            # labels = [label if label != self.processor.tokenizer.pad_token_id else -100 for label in labels]
            # all_labels.append(labels)
            
        self.targets_tokens = self.targets
        
    def __len__(self) -> int:
        return len(self.inputs)

    def __getitem__(self, idx: int):
        
        input = self.inputs[idx]
        target = self.targets_tokens
        date = self.date[idx]
                
        return input, target, date

In [169]:
data_path = Path(r"D:\Work\brain-to-text-competition\data\competitionData")

submit_dataset = BrainDataset(data_path / 'competitionHoldOut')
len(submit_dataset)

D:\Work\brain-to-text-competition\data\competitionData\competitionHoldOut\t12.2022.05.24.mat
D:\Work\brain-to-text-competition\data\competitionData\competitionHoldOut\t12.2022.05.26.mat
D:\Work\brain-to-text-competition\data\competitionData\competitionHoldOut\t12.2022.06.02.mat
D:\Work\brain-to-text-competition\data\competitionData\competitionHoldOut\t12.2022.06.07.mat
D:\Work\brain-to-text-competition\data\competitionData\competitionHoldOut\t12.2022.06.14.mat
D:\Work\brain-to-text-competition\data\competitionData\competitionHoldOut\t12.2022.06.16.mat
D:\Work\brain-to-text-competition\data\competitionData\competitionHoldOut\t12.2022.06.21.mat
D:\Work\brain-to-text-competition\data\competitionData\competitionHoldOut\t12.2022.06.28.mat
D:\Work\brain-to-text-competition\data\competitionData\competitionHoldOut\t12.2022.07.05.mat
D:\Work\brain-to-text-competition\data\competitionData\competitionHoldOut\t12.2022.07.14.mat
D:\Work\brain-to-text-competition\data\competitionData\competitionHold

1200

### Prepare submission file 

In [175]:
import string

def process_string(text):
    text = text.lower()
    punctuation = string.punctuation.replace("'", "")
    text = ''.join(char for char in text if char not in punctuation)
    return text

def make_prediction_on_dataset(submit_dataset):
    pred_list =[]
    for sample in submit_dataset:
        pred_text = "I LOVE you. don't So much AXAX"
        pred_text = process_string(pred_text)
        pred_list.append(pred_text)
    return pred_list

def create_string_file(filename, string_list):
    with open(filename, 'w') as file:
        for string in string_list:
            # Process each string
            processed_string = process_string(string)
            
            # Write the processed string to the file
            file.write(processed_string + '\n')

result = make_prediction_file(submit_dataset)
create_string_file("sub.txt", result)

In [34]:
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

In [80]:
from transformers import BertConfig, BertModel, AutoModelForCausalLM
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Download model and configuration from huggingface.co and cache.
# model = BertModel.from_pretrained("google-bert/bert-base-uncased")

In [92]:
import torch 
from transformers import AutoModelForCausalLM

decoder = AutoModelForCausalLM.from_pretrained('google-bert/bert-base-uncased', is_decoder=True, add_cross_attention=True)

Some weights of BertLMHeadModel were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['bert.encoder.layer.11.crossattention.self.value.bias', 'bert.encoder.layer.8.crossattention.output.LayerNorm.weight', 'bert.encoder.layer.3.crossattention.output.LayerNorm.weight', 'bert.encoder.layer.9.crossattention.self.key.bias', 'bert.encoder.layer.4.crossattention.self.key.weight', 'bert.encoder.layer.7.crossattention.self.value.bias', 'bert.encoder.layer.5.crossattention.self.value.weight', 'bert.encoder.layer.5.crossattention.self.value.bias', 'bert.encoder.layer.2.crossattention.self.value.weight', 'bert.encoder.layer.3.crossattention.self.value.weight', 'bert.encoder.layer.4.crossattention.self.value.bias', 'bert.encoder.layer.9.crossattention.self.query.bias', 'bert.encoder.layer.3.crossattention.self.key.bias', 'bert.encoder.layer.2.crossattention.self.value.bias', 'bert.encoder.layer.0.crossattention.self.key.bias', 'bert.encoder.laye

odict_keys(['logits', 'past_key_values'])

In [136]:
from transformers import GPT2Tokenizer, GPT2Model

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
bos = tokenizer.bos_token
eos = tokenizer.eos_token


model = GPT2Model.from_pretrained('gpt2', add_cross_attention=True)



encoder_hidden_states = torch.rand(1, 16, model.config.n_embd)

tokenizer.pad_token = '-100'
print(tokenizer)
txt = bos + 'He want you ask ' + eos

input_ids = tokenizer(txt, padding="max_length", max_length=16, add_special_tokens=True, return_tensors="pt")['input_ids']


print(input_ids)
result = model(input_ids, encoder_hidden_states=encoder_hidden_states)

Some weights of GPT2Model were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.8.crossattention.c_proj.weight', 'h.7.crossattention.c_proj.bias', 'h.11.crossattention.c_attn.bias', 'h.10.ln_cross_attn.bias', 'h.1.crossattention.c_attn.bias', 'h.10.crossattention.c_proj.weight', 'h.6.ln_cross_attn.weight', 'h.5.crossattention.c_proj.bias', 'h.8.crossattention.c_attn.weight', 'h.10.ln_cross_attn.weight', 'h.11.ln_cross_attn.weight', 'h.5.crossattention.q_attn.bias', 'h.4.ln_cross_attn.bias', 'h.6.crossattention.c_proj.bias', 'h.1.crossattention.c_proj.bias', 'h.4.crossattention.q_attn.weight', 'h.1.ln_cross_attn.weight', 'h.8.ln_cross_attn.weight', 'h.1.crossattention.c_proj.weight', 'h.3.crossattention.c_attn.bias', 'h.10.crossattention.c_proj.bias', 'h.8.crossattention.c_proj.bias', 'h.7.crossattention.q_attn.bias', 'h.11.crossattention.q_attn.weight', 'h.5.ln_cross_attn.weight', 'h.11.ln_cross_attn.bias', 'h.0.crossattention.c_attn.weight', 'h.9.crossa

GPT2Tokenizer(name_or_path='gpt2', vocab_size=50257, model_max_length=1024, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': '-100'}, clean_up_tokenization_spaces=True)
tensor([[50256,  1544,   765,   345,  1265,   220, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256]])


In [141]:
gen_tokens = model.generate(
    input_ids,
    encoder_hidden_states=encoder_hidden_states,
    do_sample=True,
    temperature=0.9,
    max_length=100,
)

TypeError: The current model class (GPT2Model) is not compatible with `.generate()`, as it doesn't have a language model head. Please use one of the following classes instead: {'GPT2LMHeadModel'}

In [154]:
encoder_hidden_states

tensor([[[0.7549, 0.8107, 0.4531,  ..., 0.9738, 0.6373, 0.2971],
         [0.6407, 0.1286, 0.6991,  ..., 0.4754, 0.1013, 0.5525],
         [0.6175, 0.4941, 0.1572,  ..., 0.5079, 0.5169, 0.7434],
         ...,
         [0.2553, 0.0449, 0.3503,  ..., 0.1883, 0.1110, 0.4542],
         [0.0213, 0.1227, 0.3695,  ..., 0.8032, 0.7929, 0.5463],
         [0.1097, 0.4807, 0.9345,  ..., 0.4903, 0.9435, 0.9352]]])

In [155]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("gpt2",  add_cross_attention=True)
tokenizer = AutoTokenizer.from_pretrained("gpt2")

bos = tokenizer.bos_token

prompt = bos

input_ids = tokenizer(prompt, return_tensors="pt").input_ids
print(input_ids)

gen_tokens = model.generate(
    input_ids,
    encoder_hidden_states=encoder_hidden_states*0,
    do_sample=False,
    temperature=0.9,
    max_length=10,
)
gen_text = tokenizer.batch_decode(gen_tokens)[0]

gen_text

Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.8.crossattention.c_proj.weight', 'h.7.crossattention.c_proj.bias', 'h.11.crossattention.c_attn.bias', 'h.10.ln_cross_attn.bias', 'h.1.crossattention.c_attn.bias', 'h.10.crossattention.c_proj.weight', 'h.6.ln_cross_attn.weight', 'h.5.crossattention.c_proj.bias', 'h.8.crossattention.c_attn.weight', 'h.10.ln_cross_attn.weight', 'h.11.ln_cross_attn.weight', 'h.5.crossattention.q_attn.bias', 'h.4.ln_cross_attn.bias', 'h.6.crossattention.c_proj.bias', 'h.1.crossattention.c_proj.bias', 'h.4.crossattention.q_attn.weight', 'h.1.ln_cross_attn.weight', 'h.8.ln_cross_attn.weight', 'h.1.crossattention.c_proj.weight', 'h.3.crossattention.c_attn.bias', 'h.10.crossattention.c_proj.bias', 'h.8.crossattention.c_proj.bias', 'h.7.crossattention.q_attn.bias', 'h.11.crossattention.q_attn.weight', 'h.5.ln_cross_attn.weight', 'h.11.ln_cross_attn.bias', 'h.0.crossattention.c_attn.weight', 'h.9.

tensor([[50256]])


'<|endoftext|>\nThe first time I saw the new version'

In [149]:
gen_text

'<|endoftext|>: The best way to get a real-life tattoo artist to do the job: Get your tattoo done in a small office in the suburbs of New York.\n\nThe best way to get a real-life tattoo artist to do the job: Get your tattoo done in a small office in the suburbs of New York. Photo: Courtesy of Sueden Tattooing\n\nKurt H. Williams, owner of Sueden Tattooing, is the owner of Sueden'

In [ ]:
def tokenize(processor, target):
    return processor.tokenizer(target, return_tensors="pt").input_ids[0]

def untokenize(processor, tokens):
    labels = tokens
    labels[labels == -100] = processor.tokenizer.pad_token_id
    label_str = processor.decode(labels, skip_special_tokens=True)
    return label_str
